In [21]:
import sys
import os 
import openmc
import numpy as np
import matplotlib.pyplot as plt
import openmc.mgxs as mgxs
import pandas as pd
from IPython.display import Image
import glob

### Import output

In [22]:
sp_file='/Users/pablogarcia44/repo/perso/openmc/17x17_assembly/vera/statepoint.50.h5'
sm_file='/Users/pablogarcia44/repo/perso/openmc/17x17_assembly/vera/summary.h5'
sm=openmc.Summary(sm_file)
sp = openmc.StatePoint(sp_file)
sp.link_with_summary(sm)
model=openmc.model.Model
model_file='/Users/pablogarcia44/repo/perso/openmc/17x17_assembly/vera/model_2H_72_families_power.xml'
model=openmc.model.Model.from_model_xml(model_file)

/opt/anaconda3/envs/openmc-env/lib/python3.12/site-packages/openmc/statepoint.py:671: RuntimeWarning: A Summary object has already been linked.
  warnings.warn('A Summary object has already been linked.',
/opt/anaconda3/envs/openmc-env/lib/python3.12/site-packages/openmc/mixin.py:70: IDWarning: Another Material instance already exists with id=13.
  warn(msg, IDWarning)
/opt/anaconda3/envs/openmc-env/lib/python3.12/site-packages/openmc/mixin.py:70: IDWarning: Another Material instance already exists with id=14.
  warn(msg, IDWarning)
/opt/anaconda3/envs/openmc-env/lib/python3.12/site-packages/openmc/mixin.py:70: IDWarning: Another Material instance already exists with id=15.
  warn(msg, IDWarning)
/opt/anaconda3/envs/openmc-env/lib/python3.12/site-packages/openmc/mixin.py:70: IDWarning: Another Material instance already exists with id=16.
  warn(msg, IDWarning)
/opt/anaconda3/envs/openmc-env/lib/python3.12/site-packages/openmc/mixin.py:70: IDWarning: Another Material instance already ex

### Get Power and Flux values

In [23]:
for index, tally in enumerate(sp.tallies):
    if index==0:
        t_power=sp.tallies[tally]
    if index==1:
        t_flux=sp.tallies[tally]
    if index==2:
        t_fission=sp.tallies[tally]

def tally_cell(cell,x,y): 
    A=[]
    cell_index = 99
    cell_id=99
    i=0
    for all_cell in model.geometry.get_all_material_cells().values():
        i=i+1
        if all_cell.name==cell+'_'+str(x)+'_'+str(y):
            cell_index=all_cell.id
            cell_id=i-1
    tally_data_reshaped =t_power.get_reshaped_data()
    power=tally_data_reshaped[cell_id]
    if x==8 or y==8:
        A.append(2*power[0][0])
    else : 
        A.append(power[0][0])
    tally_data_reshaped =t_flux.get_reshaped_data()
    flux=tally_data_reshaped[cell_id]
    A.append([flux[0][0][0],flux[1][0][0]])

    tally_data_reshaped =t_fission.get_reshaped_data()
    fission=tally_data_reshaped[cell_id]
    if x==8 or y==8:
        A.append(2*fission[0][0])
    else : 
        A.append(fission[0][0])
    return(A)

def power_cell(cell,x,y):
    return(tally_cell(cell,x,y)[0])

def flux_cell(cell,x,y):
    return(tally_cell(cell,x,y)[1])

def fission_cell(cell,x,y):
    return(tally_cell(cell,x,y)[2])



In [24]:
print(tally_cell('fuel',7,8))

print(power_cell('fuel',7,8))
print(flux_cell('fuel',7,8))
print(fission_cell('fuel',7,8))

[905381.5974366271, [0.013620050498343958, 0.057344669791170665], 0.005141625253128962]
905381.5974366271
[0.013620050498343958, 0.057344669791170665]
0.005141625253128962


### Mapping geom

In [25]:
def flux_cell(cell,x,y):    
    cell_index = 99
    cell_id=99
    i=0
    for all_cell in model.geometry.get_all_material_cells().values():
        i=i+1
        if all_cell.name==cell+'_'+str(x)+'_'+str(y):
            cell_index=all_cell.id
            cell_id=i-1
    tally_data_reshaped =t_flux.get_reshaped_data()
    flux=tally_data_reshaped[cell_id]
    flux_return=[flux[0][0][0],flux[1][0][0]]
    return(flux_return)

In [26]:
GT=[(5,2),(8,2),(3,3),(2,5),(5,5),(8,5),(2,8),(5,8)]
IT=[(8,8)]

### Mapping benchmark

In [27]:
A2=[(0,0,0.9476),(1,0,0.9418),(2,0,0.9458),(3,0,0.9551),(4,0,0.9645),(5,0,0.9739),(6,0,0.9714),(7,0,0.9721),(8,0,0.9764),(1,1,0.9389),(2,1,0.9483 ),(3,1,0.9649 ),(4,1,0.9837),(5,1,1.0115),(6,1,0.9880),(7,1,0.9880),(8,1,1.0122),(2,2,0.9736),(3,2,1.0180),(4,2,1.0360),(6,2,1.0281),(7,2,1.0266),(4,3,1.0516),(5,3,1.0458),(6,3,1.0089),(7,3,1.0053),(8,3,1.0328),(4,4,1.0313),(5,4,1.0451),(6,4,1.0118),(7,4,1.0089),(8,4,1.0353),(6,5,1.0386),(7,5,1.0368),(6,6,1.0104),(7,6,1.0093 ),(8,6,1.0371),(7,7,1.0089),(8,7,1.0364)]
B2=[(0,0,0.9478),(1,0,0.9435),(2,0,0.9475),(3,0,0.9565),(4,0,0.9662),(5,0,0.9752 ),(6,0,0.9738),(7,0,0.9742),(8,0,0.9781),(1,1,0.9406),(2,1,0.9503 ),(3,1,0.9666 ),(4,1,0.9839),(5,1,1.0099),(6,1,0.9893),(7,1,0.9893),(8,1,1.0103 ),(2,2,0.9745),(3,2, 1.0146),(4,2,1.0326),(6,2,1.0258),(7,2,1.0243),(4,3,1.0474),(5,3,1.0427),(6,3,1.0095),(7,3,1.0066),(8,3,1.0308),(4,4,  1.0308 ),(5,4,1.0424),(6,4,1.0124),(7,4,1.0099),(8,4,1.0337),(6,5,1.0373),(7,5,1.0359),(6,6,1.0121),(7,6,1.0113),(8,6,1.0362),(7,7,1.0113),(8,7,1.0355)]



B2=[(0,0,1.3188),(1,0,1.2829),(2,0,1.2384),(3,0,1.1873),(4,0,1.1405),(5,0,1.1019 ),(6,0,1.0943),(7,0,1.0862),(8,0,1.0751),(1,1,1.2416 ),(2,1,1.1840 ),(3,1,1.1144 ),(4,1,1.0495),(5,1,0.9802),(6,1,1.0104),(7,1,1.0041),(8,1,0.9609 ),(2,2,1.0905),(3,2, 0.9707),(4,2,0.9022 ),(6,2,0.8962),(7,2,0.8952),(4,3,0.8344),(5,3,0.8361),(6,3,0.9032),(7,3,0.9093),(8,3,0.8589),(4,4,0.8511),(5,4,0.8311),(6,4,0.8931),(7,4,0.9001),(8,4,0.8516),(6,5,0.8547 ),(7,5,0.8646),(6,6,0.9210),(7,6,0.9481),(8,6,0.9221),(7,7,1.0215),(8,7,1.0607)]




### Create Power RD list

In [28]:
rd_list=[]
#Create power lis
for pos in B2:
    result=power_cell('fuel',pos[0],pos[1])
    bench=pos[2]
    rd=100*(result-bench)#/bench
    rd_list.append([pos[0],pos[1],result,bench,0])
    
#Sum
sum_power=0
N=0
for k in range((len(rd_list))):
    sum_power=sum_power+rd_list[k][2]
    N=N+1
    
#Normalisation 
avg_power=sum_power/N
for k in range((len(rd_list))):
    rd_list[k][2]=rd_list[k][2]/avg_power

#Calcul rd
for pos in rd_list:
    pos[4]=100*(pos[2]-pos[3])/pos[3]
    
#Print
for k in range((len(rd_list))):
    print(rd_list[k])

[0, 0, 1.31856345354975, 1.3188, -0.01793649152639245]
[1, 0, 1.2797260029859845, 1.2829, -0.24740798300845268]
[2, 0, 1.2375423990070693, 1.2384, -0.06925072617333984]
[3, 0, 1.1834231362951946, 1.1873, -0.32652772717977296]
[4, 0, 1.1420000507130614, 1.1405, 0.13152570916802828]
[5, 0, 1.0902276562726605, 1.1019, -1.0592924700371726]
[6, 0, 1.0867359152673592, 1.0943, -0.6912258734022487]
[7, 0, 1.0804973387316013, 1.0862, -0.52501024382239]
[8, 0, 1.074655357795098, 1.0751, -0.04135821829616174]
[1, 1, 1.2430075397557245, 1.2416, 0.11336499321234667]
[2, 1, 1.1781748659680948, 1.184, -0.4919876716136064]
[3, 1, 1.1134092861017189, 1.1144, -0.08890110357871532]
[4, 1, 1.0524196252450229, 1.0495, 0.2781920195352797]
[5, 1, 0.9787674558570315, 0.9802, -0.14614814761971542]
[6, 1, 1.0042595603552993, 1.0104, -0.6077236386283332]
[7, 1, 0.9948096912781714, 1.0041, -0.9252373988475815]
[8, 1, 0.954513367849551, 0.9609, -0.6646510719584754]
[2, 2, 1.0889010333100728, 1.0905, -0.14662693167

### Verify Power Normalisation

In [29]:
test=0
N=0
for A in rd_list:
    test=test+A[2]
    N=N+1
print(test/N) 

1.0000000000000004


### Maximum rd

In [30]:
rd_max=0
x=99
y=99
for A in rd_list:
    if abs(A[4])>abs(rd_max):
        rd_max=A[4]
        x=A[0]
        y=A[1]
print(x,y,rd_max)   

5 0 -1.0592924700371726
